## Исследование надёжности заёмщиков — анализ банковских данных

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd # импорт библиотеки pandas

In [2]:
df = pd.read_csv('/datasets/data.csv') # вызов метода read_csv() и сохранение данных таблицы в переменной df. 

In [3]:
df.head(10) # печать первых 10 строк таблицы df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
df.info() # вызов метода info() для получения данных о таблице

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Определим уникальные значения в столбцах и их количество вызовом метода value_counts()

In [5]:
df['children'].value_counts() # уникальные значения и количество 'children'

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [6]:
df['dob_years'].value_counts()  # уникальные значения и количество 'dob_years'

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

In [7]:
df['income_type'].value_counts()  # уникальные значения и количество 'income_type'

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
в декрете              1
студент                1
Name: income_type, dtype: int64

In [8]:
df['education'].value_counts() #уникальные значения и количество 'education'

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

In [9]:
df['education_id'].value_counts()  #уникальные значения и количество 'education_id'

1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64

In [10]:
df['family_status'].value_counts()  #уникальные значения и количество 'family_status'

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [11]:
df['family_status_id'].value_counts() #уникальные значения и количество 'family_status_id'

0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64

In [12]:
df['gender'].value_counts() #уникальные значения и количество 'gender'

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [13]:
df['debt'].value_counts() #уникальные значения и количество 'debt'

0    19784
1     1741
Name: debt, dtype: int64

In [14]:
df['purpose'].value_counts() # уникальные значения и количество 'purpose'

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

Значения стажа в столбце 'days_employed' имеет положительные и отрицательные значения. Проверим зависит ли это от типа занятости клиентов банка. Получим строки df со значениями стажа в зависимости от типа занятости

In [15]:
df.loc[df.loc[:,'income_type'] == 'пенсионер']['days_employed']

4        340266.072047
12                 NaN
18       400281.136913
24       338551.952911
25       363548.489348
             ...      
21505    338904.866406
21508    386497.714078
21509    362161.054124
21518    373995.710838
21521    343937.404131
Name: days_employed, Length: 3856, dtype: float64

In [16]:
df.loc[df.loc[:,'income_type'] == 'безработный']['days_employed']

3133     337524.466835
14798    395302.838654
Name: days_employed, dtype: float64

In [17]:
df.loc[df.loc[:,'income_type'] == 'сотрудник']['days_employed']

0       -8437.673028
1       -4024.803754
2       -5623.422610
3       -4124.747207
7        -152.779569
            ...     
21515    -467.685130
21519   -2351.431934
21522   -2113.346888
21523   -3112.481705
21524   -1984.507589
Name: days_employed, Length: 11119, dtype: float64

In [18]:
df.loc[df.loc[:,'income_type'] == 'компаньон']['days_employed']

5        -926.185831
6       -2879.202052
10      -4171.483647
14      -1844.956182
33      -1548.637544
            ...     
21512    -165.377752
21514    -280.469996
21516    -914.391429
21517    -404.679034
21520   -4529.316663
Name: days_employed, Length: 5085, dtype: float64

In [19]:
df.loc[df.loc[:,'income_type'] == 'госслужащий']['days_employed']

26               NaN
41               NaN
47      -2689.137274
62      -7845.649233
70      -2802.226671
            ...     
21288   -2002.957310
21349   -8255.536399
21380   -2301.457587
21459    -314.968803
21507     -79.832064
Name: days_employed, Length: 1459, dtype: float64

In [20]:
df.loc[df.loc[:,'income_type'] == 'предприниматель']['days_employed']

5936            NaN
18697   -520.848083
Name: days_employed, dtype: float64

In [21]:
df.loc[df.loc[:,'income_type'] == 'студент']['days_employed']

9410   -578.751554
Name: days_employed, dtype: float64

In [22]:
df.loc[df.loc[:,'income_type'] == 'в декрете']['days_employed']

20845   -3296.759962
Name: days_employed, dtype: float64

In [23]:
df.duplicated().sum() # подсчет количества дубликатов в df, вызов метода duplicated() и sum()

54

### Вывод

Исходные данные для анализа получены и изучены.

Таблица содержит 12 столбцов. 

Названия столбцов:
* children — количество детей в семье
* days_employed — общий трудовой стаж в днях
* dob_years — возраст клиента в годах
* education — уровень образования клиента
* education_id — идентификатор уровня образования
* family_status — семейное положение
* family_status_id — идентификатор семейного положения
* gender — пол клиента
* income_type — тип занятости
* debt — имел ли задолженность по возврату кредитов
* total_income — ежемесячный доход
* purpose — цель получения кредита

Данные в столбцах трех типов: float64 (2 столбца), int64 (5 столбцов), object (5 столбцов).

Исходная таблица df содержит 54 дубликата.

Количество ненулевых значений в столбцах 'days_employed' и  'total_income' отличается от остальных столбцов. Они содержат пропущенные значения.

Возможная причина пропусков: клиенты банка не подтвердили информацию о своем стаже и ежемесячном доходе (отсутствие справки о доходах с работы (справка 2-НДФЛ или справка о доходе по форме банка).

Также данные содержат следующие проблемы:

1) в столбце 'children' 47 строк с отрицательным значением количества детей (-1) и 76 строк со значением 20. 

Возможная причина: опечатка при заполнении данных.

2) в столбце 'days_employed' значения стажа отрицательные у всех, кроме пенсионеров и безработных. Возможная причина: в банке принята такая форма записи стажа. Эти данные не критичны для решения поставленной задачи.

3) у пенсионеров и безработных - очень большие значения стажа, эти данные приведены не в днях. Возможная причина: для пенсионеров и безработных в банке принята такая форма записи стажа. Эти данные не критичны для решения поставленной задачи.

4) в столбце 'dob_years' содержится 101 значение с возрастом '0'. Возможная причина: сотрудник банка пропустил и не заполнил эти данные. Эти данные не критичны для решения поставленной задачи.

5) в столбце 'education' значения строк отличаются регистром букв. Причина: данные заполняются вручную. Эти данные не критичны для решения поставленной задачи, но при подсчете дубликатов значения с разным регистром будут посчитаны уникальными.

6) в столбце 'gender' у одной строки не определен пол (XNA). Возможная причина: данные не были заполнены. Эти данные не критичны для решения поставленной задачи.

7) в столбце 'purpose' похожие цели представлены по разному. Причина: цель кредита заполняется вручную.

### Шаг 2. Предобработка данных

### Обработка пропусков

Определение количества пропусков в df

In [24]:
df.isnull().sum() # проверка данных на наличие пропусков вызовом методов isnull() и sum() для суммирования пропущенных значений

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Количество данных с пропусками составляет 10% от всех данных, поэтому просто удалить их нельзя.

Пропуски в 'total_income' заменим на значение медианы в зависимости от 'income_type'

Пропуски в 'days_employed' заменим на значение медианы в зависимости от 'dob_years'

In [25]:
df['income_type'].unique() # получим список со всеми уникальными значениями 'income_type'

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [26]:
income_type_list = ['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'] # сохраним список в переменную income_type_list

In [27]:
# функция для замены пропущенных значений в 'total_income' для каждой 'income_type' на значение медианы
# сохраним логический индекс в переменную income_type_index
# income_median - определяем значение медианы для каждого 'income_type'
# заменяем пропуски в 'total_income' на значение медианы для каждого 'income_type'

def replace_income_median(data, income_type):
    income_type_index = data['income_type'] == income_type
    income_median = data[income_type_index]['total_income'].median() 
    data.loc[income_type_index,'total_income'] = data[income_type_index]['total_income'].fillna(value= income_median)

In [28]:
for income_type in income_type_list:
    replace_income_median(df, income_type)

In [29]:
df['total_income'].isnull().sum() # проверка на наличие пропусков в 'total_income' 

0

In [30]:
df['dob_years'].unique() # получим список со всеми уникальными значениями 'dob_years'

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75])

In [31]:
dob_years_list = [42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75] # сохраним список в переменную dob_years_list

In [32]:
#функция замены пропущенных значений в 'days_employed' для каждой 'dob_years' на значение медианы
# сохраним логический индекс в переменную dob_years_index
# days_employed_median - определяем значение медианы для каждого 'dob_years'
# заменяем пропуски в 'days_employed' на значение медианы для каждого 'dob_years'

def replace_days_employed_median(data, dob_years):
    dob_years_index = data['dob_years'] == dob_years
    days_employed_median = data[dob_years_index]['days_employed'].median() 
    data.loc[dob_years_index,'days_employed'] = data[dob_years_index]['days_employed'].fillna(value= days_employed_median)

In [33]:
for dob_years in dob_years_list:
    replace_days_employed_median(df, dob_years)

In [34]:
df['days_employed'].isnull().sum()

0

В данных о количестве детей имеется два ошибочных значения (-1) и 20. Можно предположить, что эти анкетные данные заполнялись работником банка вручную и была допущена опечатка: (-1) это 1, а 20 - это на самом деле 2.

Заменим (-1) на 1, 20 на 2 методом replace()

In [35]:
df['children'] = df['children'].replace(-1, 1) # вызов метода replace, замена -1 на 1 в столбце 'children' таблицы df

In [36]:
df['children'] = df['children'].replace(20, 2) # вызов метода replace, замена 20 на 2 в столбце 'children' таблицы df

In [37]:
df['children'].value_counts() # проверка уникальных значений children

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

### Вывод

В исходной таблице df пропуски значений есть в двух столбцах: 'days_employed' и 'total_income'. Количество пропусков равно 2174, что составляет 10% от всех данных, поэтому просто удалить их нельзя.

Возможная причина пропусков: клиенты банка не подтвердили информацию о своем стаже и ежемесячном доходе (отсутствие справки о доходах с работы (справка 2-НДФЛ или справка о доходе по форме банка)) и поля остались незаполненными. 

Пропуски в 'total_income' заменены на значение медианы в зависимости от 'income_type'. Пропуски в 'days_employed' заменены на значение медианы в зависимости от 'dob_years'.

Ошибки в данных о количестве детей исправленны исходя из предположения, что (-1) это 1, а 20 на самом деле 2.

### Замена типа данных

Данные в таблице df в столбцах 'days_employed' и 'total_income' имеют вещественный тип данных. Заменим в df в столбцах 'days_employed' и 'total_income' вещественный тип данных на целочисленный, для этого вызовем метод astype()

In [38]:
df['days_employed'] = df['days_employed'].astype('int') # вызов метода astype() для замены типа данных float64 на int64

In [39]:
df['total_income'] = df['total_income'].astype('int')

In [40]:
df.info() # проверка типа данных в df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Вывод

Данные таблицы df столбцы  'days_employed' и 'total_income' приведены к целочисленному типу данных, методом astype().

### Обработка дубликатов

In [41]:
df.duplicated().sum() # вызов методов duplicated() и sum() - проверка отсутствия дудликатов в df_income

54

Подсчитаем долю дубликатов от количества строк в таблице df.

In [42]:
'{:.1%}'.format(df.duplicated().sum() / df.shape[0])

'0.3%'

Количество дубликатов осталось как и у исходной таблицы - 54. Значения всех полей таких дубликатов одинаковы и представляют из себя избыточные, не уникальные данные. Процент этих данных мал (~0.3%) и мы имеем право их удалить.

Удалим 54 дубликата из таблицы df со сбросом индексов

In [43]:
df = df.drop_duplicates().reset_index(drop = True) # вызов метода drop_duplicates() для удаления дубликатов
                                                   # вызов метода reset_index() для сброса индексов

In [44]:
df.duplicated().sum() # проверка на дубликаты в df

0

In [45]:
df.info() # проверка типа данных в df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21471 entries, 0 to 21470
Data columns (total 12 columns):
children            21471 non-null int64
days_employed       21471 non-null int64
dob_years           21471 non-null int64
education           21471 non-null object
education_id        21471 non-null int64
family_status       21471 non-null object
family_status_id    21471 non-null int64
gender              21471 non-null object
income_type         21471 non-null object
debt                21471 non-null int64
total_income        21471 non-null int64
purpose             21471 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Вывод

В таблице df обнаружено 54 дубликата (~ 0,3% данных), ими можно пренебречь. Дубликаты удалены со сбросом индексов.

Возможные причинины появления дубликатов: 
* повторный ввод данных сотрудником банка;
* ошибка при выгрузке данных.


### Лемматизация

Лемматизация необходима для того, чтобы преобразовать неформализованные текстовые данные, введенные вручную в более упорядоченный и фиксированный набор строковых значений, по которым удобно делать группировки для последующего анализа и выводов.

In [46]:
from pymystem3 import Mystem # импорт библиотеки pymystem3
m = Mystem()

In [47]:
df['purpose_lemma'] = df['purpose'].apply(m.lemmatize)

Выполним подсчет числа упоминаний в df лемматизированных слов. Для этого вызовем специальный контейнер Counter из модуля collections.

In [48]:
from collections import Counter

In [49]:
print(Counter(df['purpose_lemma'].sum()))

Counter({' ': 33596, '\n': 21471, 'недвижимость': 6353, 'покупка': 5900, 'жилье': 4461, 'автомобиль': 4308, 'образование': 4014, 'с': 2918, 'операция': 2604, 'свадьба': 2335, 'свой': 2231, 'на': 2228, 'строительство': 1879, 'высокий': 1374, 'получение': 1315, 'коммерческий': 1312, 'для': 1290, 'жилой': 1231, 'сделка': 941, 'дополнительный': 907, 'заниматься': 904, 'проведение': 773, 'сыграть': 769, 'сдача': 652, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'подержанный': 486, 'подержать': 478, 'приобретение': 461, 'профильный': 436})


In [50]:
test = Counter(df['purpose_lemma'].sum())

In [51]:
test.most_common()

[(' ', 33596),
 ('\n', 21471),
 ('недвижимость', 6353),
 ('покупка', 5900),
 ('жилье', 4461),
 ('автомобиль', 4308),
 ('образование', 4014),
 ('с', 2918),
 ('операция', 2604),
 ('свадьба', 2335),
 ('свой', 2231),
 ('на', 2228),
 ('строительство', 1879),
 ('высокий', 1374),
 ('получение', 1315),
 ('коммерческий', 1312),
 ('для', 1290),
 ('жилой', 1231),
 ('сделка', 941),
 ('дополнительный', 907),
 ('заниматься', 904),
 ('проведение', 773),
 ('сыграть', 769),
 ('сдача', 652),
 ('семья', 638),
 ('собственный', 635),
 ('со', 627),
 ('ремонт', 607),
 ('подержанный', 486),
 ('подержать', 478),
 ('приобретение', 461),
 ('профильный', 436)]

Таким образом, можно выделить 4 цели - недвижимость, свадьба, образование, автомобиль.

Дополнительно, чтобы еще раз проверить, что цели определены правильно, можно получить все уникальные значения столбца 'purpose' и их количество, методом value_counts(). В результате будет получен полный список всех возможных целей, которые указывали клиенты банка. Этих данные не много и в данном конкретном случае их можно просмотреть глазами, и убедиться что цели (недвижимость, свадьба, образование, автомобиль) определены верно.

(Доп. уточнение: под недвижимостью будем понимать любые операции связанные с жильем, недвижимостью, коммерческой недвижимость и т.п.)

In [52]:
df['purpose'].value_counts()

свадьба                                   793
на проведение свадьбы                     773
сыграть свадьбу                           769
операции с недвижимостью                  675
покупка коммерческой недвижимости         662
операции с жильем                         652
покупка жилья для сдачи                   652
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          625
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

In [54]:
#Функция для определения конечной цели кедита (недвижимость / свадьба / образование / автомобиль)

def alter_purpose_new(row):
    alter_purpose = row['purpose_lemma']
    
    if ('жилье' in alter_purpose) or ('недвижимость' in alter_purpose):
        return 'недвижимость'
    
    if 'свадьба' in alter_purpose:
        return 'свадьба'
    
    if 'образование' in alter_purpose:
        return 'образование'
    
    if 'автомобиль' in alter_purpose:
        return 'автомобиль'

In [55]:
df['purpose_new'] = df.apply(alter_purpose_new, axis=1) # вызов метода apply(), создание в таблице df столбца 'purpose_new'

In [56]:
df['purpose_new'].value_counts() # проверка

недвижимость    10814
автомобиль       4308
образование      4014
свадьба          2335
Name: purpose_new, dtype: int64

### Вывод

Определены цели кредитов. Всего 4 категории:

1) недвижимость
2) автомобиль
3) образование
4) свадьба

Лидером среди кредитов является кредит на недвижимость (50% всех кредитов в банке). Это связано в высокой стоимостью жилья и, соответственно, люди чаще берут кредит на эти цели.

Кредит на автомобиль (20%) и образование (19%) берут более, чем в 2 раза реже, по сравнению с кредитом на недвижимость. Эти две цели примерно одинаковые по популярности.

Кредит на свадьбу не такой популярный, его берут только 11% клиентов банка. Это может быть связано с тем, что уровень расходов на свадьбу у всех разный и чаще всего людям хватает собственных средств для проведения свадьбы.

### Категоризация данных

Перед категоризацией данных создадим новые столбцы на основе данных из других столбцов, вызовом метода apply(): 

* новый столбец 'income_group' из 'total_income'
* новый столбец 'children_group' из 'children'
* новый столбец 'family_status_group' из 'family_status_id'

Для разделения данных по уровню дохода используем метод describe() и получим основную информацию по столбцу 'total_income', в том числе 25, 50 и 75 процентили.
Процентиль показывает, какая доля клиентов банка имеет доход ниже данного уровня. 

Выделим следующие категории по доходу:
* менее 25% - это клиенты банка, которые имеют доход ниже уровня, равного 25-ой процентили.
* от 25% до 50% - это клиенты банка, которые имеют доход больше или равно 25-ой процентили, но меньше 50-ой процентили.
* от 50% до 75% - это клиенты банка, которые имеют доход больше или равно 50-ой процентили, но меньше 75-ой процентили.
* более 75% - это клиенты банка, которые имеют доход больше 75-ой процентили.

In [57]:
percent = df['total_income'].describe() # вызов метода describe()

In [58]:
percent

count    2.147100e+04
mean     1.652954e+05
std      9.815346e+04
min      2.066700e+04
25%      1.076545e+05
50%      1.425940e+05
75%      1.957675e+05
max      2.265604e+06
Name: total_income, dtype: float64

In [59]:
perc_25 = percent['25%'] # сохранение значения 25 процентиля в переменную perc_25

In [60]:
perc_50 = percent['50%'] # сохранение значения 50 процентиля в переменную perc_50

In [61]:
perc_75 = percent['75%'] # сохранение значения 75 процентиля в переменную perc_75

In [62]:
def income_group(total_income):
    
    if total_income < perc_25:
        return 'менее 25%'
    
    if (total_income >= perc_25) & (total_income < perc_50):
        return 'от 25% до 50%'
    
    if (total_income >= perc_50) & (total_income < perc_75):
        return 'от 50% до 75%'
    
    return 'более 75%'

In [63]:
df['income_group'] = df['total_income'].apply(income_group) # вызов метода apply(), добавление в таблицу df_income столбца 'income_group'

In [64]:
df['income_group'].value_counts()

от 50% до 75%    6322
менее 25%        5368
более 75%        5368
от 25% до 50%    4413
Name: income_group, dtype: int64

Категоризация по наличию и количеству детей. 

Клиентов разделим на категории по факту наличия детей:

* нет детей
* есть дети

А также сгруппируем клиентов по количеству детей:
* 1 ребенок
* 2 ребенок
* 3 и более детей (многодетные семьи)

In [65]:
def children_group_exist(children):
    
    if children >= 1:
        return 'есть дети'
    
    return 'нет детей'

In [66]:
df['children_group_exist'] = df['children'].apply(children_group_exist) # вызов метода apply(), добавление в df столбца 'children_group_exist'

In [67]:
df['children_group_exist'].value_counts() # проверка

нет детей    14107
есть дети     7364
Name: children_group_exist, dtype: int64

In [68]:
def children_group_number(children):
    
    if children == 0:
        return 'нет детей'
    
    if children == 1:
        return '1 ребенок'
    
    if children == 2:
        return '2 ребенка'
    
    return '3 и более детей'

In [69]:
df['children_group_number'] = df['children'].apply(children_group_number) # вызов метода apply(), добавление в df столбца 'children_group_number'

In [70]:
df['children_group_number'].value_counts() # проверка

нет детей          14107
1 ребенок           4856
2 ребенка           2128
3 и более детей      380
Name: children_group_number, dtype: int64

Категоризация по семейному положению. Определим факт наличия брака (в т.ч. гражданского) и факт отсутствия брака. 

In [71]:
def family_status_group(family_status_id):
    
    if (family_status_id == 0) or (family_status_id == 1):
        return 'в браке'
    
    return 'не в браке'

In [72]:
df['family_status_group'] = df['family_status_id'].apply(family_status_group) # вызов метода apply(), добавление в df столбца 'family_status_group'

In [73]:
df['family_status_group'].value_counts() # проверка

в браке       16507
не в браке     4964
Name: family_status_group, dtype: int64

### Вывод

Выполнена категоризация данных по:

* уровню дохода (менее 25% / от 25% до 50% / от 50% до 75% / более 75%);
* факту наличия детей (есть дети / нет детей) и количества детей (1 ребенок / 2 ребенка / 3 ребенка и более / нет детей); 
* семейному положению (в браке / не в браке).

Среди клиентов банка: 
* 77% находятся в браке;
* 66% не имеют детей.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Построим сводные таблицы:

In [74]:
df_pivot_children_group = df.pivot_table(index=['children_group_exist'], columns='debt', 
                                               values='total_income', aggfunc='count', fill_value=0) 
                                                # вызыв метод pivot_table(), построение сводной таблицы

In [75]:
df_pivot_children_group['share'] = df_pivot_children_group[1] / (df_pivot_children_group[1] + df_pivot_children_group[0])

In [77]:
df_pivot_children_group.sort_values(by ='share', ascending = True) # печать сводной таблицы, 
                                                                   # отсортированной по столбцу 'share' по возрастанию

debt,0,1,share
children_group_exist,,,
нет детей,13044,1063,0.075353
есть дети,6686,678,0.092070


In [400]:
df_pivot_children = df.pivot_table(index=['children_group_number'], columns='debt', 
                                                values='total_income', aggfunc='count', fill_value=0) 
                                                # вызыв метод pivot_table(), построение сводной таблицы

In [401]:
df_pivot_children['share'] = df_pivot_children[1] / (df_pivot_children[1] + df_pivot_children[0])

In [402]:
df_pivot_children.sort_values(by ='share', ascending = True) # печать сводной таблицы, 
                                                            # отсортированной по столбцу 'share' по возрастанию

debt,0,1,share
children_group_number,,,
нет детей,13044,1063,0.075353
3 и более детей,349,31,0.081579
1 ребенок,4411,445,0.091639
2 ребенка,1926,202,0.094925


### Вывод

Построены две сводные тоблицы зависимости факта наличия детей (наличия детей и их количества) и наличия задолженности по кредиту.

Среди тех, кто имеет детей, доля тех, кто имеет задолженность по кредиту составляет 9,2%.
Для тех, у кого нет детей, доля тех, кто имеет задолженность по кредиту составляет 7,5%.

Количество детей не влияет положительно на показатель возврата кредита в срок.
Среди клиентов с 1 ребенком задолженность имеют 9,2%
Клиенты с 2 детьми - 9,5%
Клиенты с 3 детьми и более - 8,2%

Можно сделать вывод, что факта наличия детей ухудшает показатель возврата кредита в срок. Клиенты без детей имеют меньше задолженностей перед банком.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Построим сводные таблицы:

In [403]:
df_pivot_family_status_group = df.pivot_table(index=['family_status_group'], columns='debt', 
                                              values='total_income', aggfunc='count', fill_value=0)
                                              # вызыв метод pivot_table(), построение сводной таблицы

In [404]:
df_pivot_family_status_group['share'] = df_pivot_family_status_group[1]/(df_pivot_family_status_group[1] + df_pivot_family_status_group[0])

In [405]:
df_pivot_family_status_group.sort_values(by ='share', ascending = True) # печать сводной таблицы, 
                                                                        # отсортированной по столбцу 'share' по возрастанию

debt,0,1,share
family_status_group,,,
в браке,15188,1319,0.079905
не в браке,4542,422,0.085012


In [406]:
df_pivot_family_status = df.pivot_table(index=['family_status_group','family_status'], columns='debt', 
                                        values='total_income', aggfunc='count', fill_value=0)
                                        # вызыв метод pivot_table(), построение сводной таблицы

In [407]:
df_pivot_family_status['share'] = df_pivot_family_status[1]/(df_pivot_family_status[1] + df_pivot_family_status[0])

In [408]:
df_pivot_family_status # печать сводной таблицы

debt                                           0    1     share
family_status_group family_status                              
в браке             гражданский брак        3775  388  0.093202
                    женат / замужем        11413  931  0.075421
не в браке          Не женат / не замужем   2536  274  0.097509
                    в разводе               1110   85  0.071130
                    вдовец / вдова           896   63  0.065693

### Вывод

Построены две сводные таблицы зависимости семейного положения и наличия задолженности по кредиту.

Среди клиентов банка, которые находятся в браке, доля тех, кто имеет задолженность по кредиту составляет 8,0%.
Среди клиентов банка, которые не в браке, доля тех, кто имеет задолженность по кредиту составляет 8,5%.

Также среди тех, кто в браке, меньше всего задолженностей по кредиту у тех, кто находится в официальном браке (7,5%) по сравнению с гражданским браком (9,3%).

Среди тех, кто не в браке, самый высокий процент факта задолженности у тех, кто имеет статус "Не женат / не замужем" (9,8%), меньше всего у категории "вдовец / вдова" (6,6%).

В целом, клиенты в браке и не в браке имеют примерно одинаковый процент задолженности по кредитам. 
Среди тех, кто в браке, лучший показатель по возврату кредита у клиентов, которые находятся в официальном браке.
Среди клиентов, которые не в браке, хуже всего показатель по возврату кредита у клиентов, которые имеют статус не женат/не замужем.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Построим сводную таблицу:

In [409]:
df_pivot_income = df.pivot_table(index=['income_group'], columns='debt', 
                                        values='total_income', aggfunc='count', fill_value=0)
                                        # вызыв метод pivot_table(), построение сводной таблицы

In [410]:
df_pivot_income['share'] = df_pivot_income[1] / (df_pivot_income[1] + df_pivot_income[0])

In [411]:
df_pivot_income.sort_values(by ='share', ascending = True) # печать сводной таблицы, 
                                                            # отсортированной по столбцу 'share' по возрастанию

debt,0,1,share
income_group,,,
более 75%,4985,383,0.071349
менее 25%,4941,427,0.079545
от 50% до 75%,5776,546,0.086365
от 25% до 50%,4028,385,0.087242


### Вывод

Построена сводная таблица зависимости дохода и наличия задолженности по кредиту.

Для клиентов с уровнем дохода менее 25% доля задолженности по кредиту составляет 8,0%.
Для клиентов с уровнем дохода больше или равно 25% и меньше 50% - 8,7%.
Для клиентов с уровнем дохода больше или равно 50% и меньше 75% - 8,6%.
Для клиентов с уровнем дохода более 75% - 7,1%.

Самым низкий процент задолженности по кредитам у клиентов с уровнем дохода, более 75%.
Клиенты с доходом от 25-50% и 50-75% имеют примерно одинаковые и наиболее высокие показатели невозврата по кредитам.

Анализ по уровню дохода показал, что нет линейной зависимости показателя невозврата по кредитам от роста величины дохода. При этом, клиенты с доходом более 75%  имеет лучший показатель по возврату кредита в срок.

- Как разные цели кредита влияют на его возврат в срок?

Построим свобную таблицу:

In [412]:
df_pivot_purpose = df.pivot_table(index=['purpose_new'], columns='debt', 
                                  values='total_income', aggfunc='count', fill_value=0)
                                  # вызыв метод pivot_table(), построение сводной таблицы

In [413]:
df_pivot_purpose['share'] = df_pivot_purpose[1] / (df_pivot_purpose[1] + df_pivot_purpose[0])

In [414]:
df_pivot_purpose.sort_values(by ='share', ascending = True) # печать сводной таблицы, 
                                                            # отсортированной по столбцу 'share' по возрастанию

debt,0,1,share
purpose_new,,,
недвижимость,10032,782,0.072314
свадьба,2149,186,0.079657
образование,3644,370,0.092177
автомобиль,3905,403,0.093547


### Вывод

Построена сводная таблица зависимости цели кредита и наличия задолженности по кредиту.

Клиенты, которые берут кредит на недвижимость, имеют меньше всего задолженностей перед банком (7,2%).

Кредит на свадьбы не возвращает 8,0% клиентов банка.

Хуже всего клиенты банка возвращают кредиты на автомобили (9,4%) и образование (9,2%).

### Шаг 4. Общий вывод

Выполнен анализ влияния семейного положения, количества детей, дохода и целей кредита на факт погашения кредита в срок.

Меньше задолженностей по кредитам у следующих клиентов:

* Клиенты в браке;
* Клиенты без детей;
* Клиенты с доходом более 75%;
* Клиенты, которые берут кредит на операции с недвижимостью. 